In [ ]:
# default_exp local_runner

In [ ]:
#hide
print(get_notebook_path())

/usr/local/google/home/alekseyv/vlasenkoalexey/gcp_runner/local_runner.ipynb


In [ ]:
#export
from gcp_runner.core import get_run_python_args, run_process

def run_python(func, python_binary='python', **kwargs):
    args = get_run_python_args(func, python_binary, **kwargs)
    print('Running as python script:')
    print(' '.join(args))
    run_process(args)

In [ ]:
from gcp_runner.core import export_and_reload_all
export_and_reload_all(silent=True)

In [ ]:
#export
import os
from gcp_runner.core import get_run_python_args, run_process, get_package_name, build_and_push_docker_image

def run_docker(func, image_uri, build_docker_file=None, push_docker=False, docker_args=None, dry_run=False, **kwargs):
    if build_docker_file is not None:
        result = build_and_push_docker_image(build_docker_file, image_uri, push_docker, dry_run=dry_run)
        if result:
            return result
    
    package_name = get_package_name()
    args = ['docker', 'run', '-v', 
            os.getcwd() + '/%s:/%s' % (package_name, package_name)]
    if docker_args:
        args.extend(docker_args)
    args.append(image_uri)
    run_python_args = get_run_python_args(func, **kwargs)
    args.extend(run_python_args)
    print('Running in Docker container:')
    print(' '.join(args))
    if not dry_run:
        return run_process(args) 
    return 0
    

In [ ]:
#export
from gcp_runner.core import get_run_python_args, run_process, get_package_name, get_module_name

#IMPORTANT: we'll need to either copy entry_point.py to package,
# or tell user to declare it and call into similar entry_point logic manually.
#
# !!! Doesn't work because of https://github.com/GoogleCloudPlatform/cloudml-samples/issues/476
def run_on_ai_platform(func, job_dir):
    package_name = get_package_name()
    module_name = get_module_name()
    function_name = func.__name__
    args = ['gcloud', 'ai-platform', 'local', 'train',
           "--job-dir=%s" % job_dir,
           "--module-name=%s.entry_point" % package_name,
           "--package-path=%s/%s " % (os.getcwd(), package_name),
           "--",
           "--job-dir=%s" % job_dir, 
           "--module-name=%s.%s" % (package_name, module_name),
           "--function-name=%s" % function_name]
    print('Running training job using local Cloud AI:')
    print(' '.join(args).replace(' --', '\n --').replace('\n', ' \\ \n'))
    run_process(args)
    

In [ ]:
from nbdev.export import *
notebook2script()


Converted 00_core.ipynb.
Converted 01_ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code.ipynb.
